In [1]:
import os
import sys
sys.path.insert(0, os.path.abspath('..'))
import uuid

from autocnet.examples import get_path
from autocnet.graph.network import CandidateGraph, Node, Edge
from autocnet.graph.edge import Edge
from autocnet.matcher.feature import FlannMatcher

from plio.io import io_autocnetgraph

import numpy as np
import pandas as pd

import farmhash

%pylab inline

CUDA Disabled
Populating the interactive namespace from numpy and matplotlib


## Generate a 2 image adjacenecy graph

In [2]:
#Point to the adjacency Graph
adjacency = get_path('three_image_adjacency.json')
basepath = get_path('Apollo15')
cg = CandidateGraph.from_adjacency(adjacency, basepath=basepath)

#Apply SIFT to extract features
cg.extract_features(method='sift', extractor_parameters={'nfeatures':50})

#Match
cg.match()

#Apply outlier detection
cg.symmetry_checks()
cg.ratio_checks()
cg.compute_fundamental_matrices()
cg.compute_homographies()
cg.save('example_save.project')

/Users/jlaura/github/autocnet/autocnet/transformation/fundamental_matrix.py:193: UserWarning: Unable to apply MLE.  Not enough correspondences.  Returning with a RANSAC computed F matrix.
  warnings.warn("Unable to apply MLE.  Not enough correspondences.  Returning with a RANSAC computed F matrix.")


In [3]:
from plio.io.io_autocnetgraph import load
rebuilt = load('example_save.project')

In [4]:
cg == rebuilt

True

In [92]:
from io import BytesIO
from zipfile import ZipFile

from networkx.readwrite import json_graph
import base64
import json
import h5py

class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        """If input object is an ndarray it will be converted into a dict 
        holding dtype, shape and the data, base64 encoded.
        """
        if isinstance(obj, np.ndarray):
            return dict(__ndarray__= obj.tolist(),
                        dtype=str(obj.dtype),
                        shape=obj.shape)
        elif isinstance(obj, uuid.UUID):
            return obj.hex
        # Let the base class default method raise the TypeError
        return json.JSONEncoder.default(self, obj)

def save(network, projectname):
    """
    Save an AutoCNet candiate graph to disk in a compressed file.  The
    graph adjacency structure is stored as human readable JSON and all
    potentially large numpy arrays are stored as compressed binary. The
    project archive is a standard .zip file that can have any ending,
    e.g., <projectname>.project, <projectname>.zip, <projectname>.myname.
    
    TODO: This func. writes a intermediary .npz to disk when saving.  Can
    we write the .npz to memory?
    
    Parameters
    ----------
    network : object
              The AutoCNet Candidate Graph object
    
    projectname : str
                  The PATH to the output file.
    """
    # Convert the graph into json format
    js = json_graph.node_link_data(network) 
    
    with ZipFile(projectname, 'w') as pzip:
        js_str = json.dumps(js, cls=NumpyEncoder, sort_keys=True, indent=4)
        pzip.writestr('graph.json', js_str)

        # Write the array node_attributes to hdf
        for n, data in network.nodes_iter(data=True):
            grp = data['node_id']
            np.savez('{}.npz'.format(data['node_id']),
                     descriptors=data.descriptors,
                     _keypoints=data._keypoints,
                     _keypoints_idx=data._keypoints.index,
                     _keypoints_columns=data._keypoints.columns)
            pzip.write('{}.npz'.format(data['node_id']))
            os.remove('{}.npz'.format(data['node_id']))

        # Write the array edge attributes to hdf
        for s, d, data in network.edges_iter(data=True):
            if s > d:
                s, d = d, s
            grp = str((s,d))
            np.savez('{}_{}.npz'.format(s, d),
                     matches=data.matches,
                     matches_idx=data.matches.index,
                     matches_columns=data.matches.columns,
                     _masks=data._masks,
                     _masks_idx=data._masks.index,
                     _masks_columns=data._masks.columns)
            pzip.write('{}_{}.npz'.format(s, d))
            os.remove('{}_{}.npz'.format(s, d))
save(cg, 'sampleout.project')

In [93]:
!ls -alh

total 7232
drwxr-xr-x  16 jlaura  flagstaf   544B Feb  2 09:52 .
drwxr-xr-x  37 jlaura  flagstaf   1.2K Feb  2 09:14 ..
-rw-r--r--@  1 jlaura  flagstaf   8.0K Feb  2 09:29 .DS_Store
drwxr-xr-x   9 jlaura  flagstaf   306B Jan 11 19:53 .ipynb_checkpoints
-rw-r--r--   1 jlaura  flagstaf   4.9K Jan 11 19:48 Ciratefi.ipynb
-rw-r--r--   1 jlaura  flagstaf   1.3M Dec 16 20:55 Coupled+Decomposition.ipynb
-rw-r--r--   1 jlaura  flagstaf    13K Dec  3 13:14 ISIS Serial Numbers.ipynb
-rw-r--r--   1 jlaura  flagstaf    26K Feb  2 09:52 Saving.ipynb
-rw-r--r--   1 jlaura  flagstaf   7.1K Jan 11 19:48 Suppression vis Disk Covering.ipynb
-rw-r--r--   1 jlaura  flagstaf   9.7K Jan 31 07:41 Testing Cuda Integration.ipynb
-rw-r--r--   1 jlaura  flagstaf    24K Dec  3 13:14 Testing Slopes.ipynb
-rw-r--r--   1 jlaura  flagstaf   1.4M Jan 11 19:48 Tutorial with Visualization.ipynb
-rw-r--r--   1 jlaura  flagstaf   453K Jan 11 19:49 Voronoi.ipynb
drwx------   9 jlaura  flagstaf   306B Feb  2 09:29 sampleout

In [94]:
def json_numpy_obj_hook(dct):
    """Decodes a previously encoded numpy ndarray with proper shape and dtype.

    :param dct: (dict) json encoded ndarray
    :return: (ndarray) if input was an encoded ndarray
    """
    if isinstance(dct, dict) and '__ndarray__' in dct:
        data = np.asarray(dct['__ndarray__'])
        return np.frombuffer(data, dct['dtype']).reshape(dct['shape'])
    return dct

def load(projectname):
    with ZipFile(projectname, 'r') as pzip:
        # Read the graph object
        with pzip.open('graph.json', 'r') as g:
            data = json.loads(g.read().decode(),object_hook=json_numpy_obj_hook)
        
        cg = CandidateGraph()
        # Reload the graph attributes
        cg.graph = data['graph']
        # Handle nodes
        for d in data['nodes']:
            n = Node(image_name=d['image_name'], image_path=d['image_path'], node_id=d['id'])
            n['hash'] = d['hash']
            # Load the byte stream for the nested npz file into memory and then unpack
            nzf = np.load(BytesIO(pzip.read('{}.npz'.format(d['id']))))
            n._keypoints = pd.DataFrame(nzf['_keypoints'], index=nzf['_keypoints_idx'], columns=nzf['_keypoints_columns'])
            n.descriptors = nzf['descriptors']
            cg.add_node(d['node_id'])
            cg.node[d['node_id']] = n
        for e in data['links']:
            cg.add_edge(e['source'], e['target'])
            edge = Edge()
            edge.source = cg.node[e['source']]
            edge.destination = cg.node[e['target']]
            edge['fundamental_matrix'] = e['fundamental_matrix']
            edge['weight'] = e['weight']
            nzf = np.load(BytesIO(pzip.read('{}_{}.npz'.format(e['source'], e['target']))))

            edge._masks = pd.DataFrame(nzf['_masks'], index=nzf['_masks_idx'], columns=nzf['_masks_columns'])
            edge.matches = pd.DataFrame(nzf['matches'], index=nzf['matches_idx'], columns=nzf['matches_columns'])
            # Add a mock edge
            cg.edge[e['source']][e['target']] = edge

    return cg

rebuilt = load('sampleout.project')


In [91]:
cg == rebuilt

True

In [13]:
rebuilt.edge[0][1].matches['source_image'] == 0.

0      True
1      True
2      True
3      True
4      True
5      True
6      True
7      True
8      True
9      True
10     True
11     True
12     True
13     True
14     True
15     True
16     True
17     True
18     True
19     True
20     True
21     True
22     True
23     True
24     True
25     True
26     True
27     True
28     True
29     True
       ... 
170    True
171    True
172    True
173    True
174    True
175    True
176    True
177    True
178    True
179    True
180    True
181    True
182    True
183    True
184    True
185    True
186    True
187    True
188    True
189    True
190    True
191    True
192    True
193    True
194    True
195    True
196    True
197    True
198    True
199    True
Name: source_image, dtype: bool

In [57]:
df = pd.DataFrame(cg.edge[0][1].matches.values, columns=['a', 'b', 'c', 'd', 'e'],
                  dtype=cg.edge[0][1].matches.dtypes)
df.dtypes
df = df.apply(pd.to_numeric)
df.dtypes

a    float64
b    float64
c    float64
d    float64
e    float64
dtype: object

In [48]:
np.asarray(cg.edge[0][1].matches.dtypes)

array([dtype('int64'), dtype('int64'), dtype('int64'), dtype('int64'),
       dtype('float64')], dtype=object)

In [22]:
rebuilt.edge[0][1].matches.dtypes = cg.edge[0][1].matches.dtypes

AttributeError: can't set attribute

In [14]:
rebuilt.edge[0][2].matches[(rebuilt.edge[0][2].matches != cg.edge[0][2].matches).any(axis=1)]

,source_image,source_idx,destination_image,destination_idx,distance


In [17]:
cg.edge[0][1].matches['source_image'].dtype

dtype('int64')

In [19]:
rebuilt.edge[0][1].matches['source_image'].dtype

dtype('float64')

In [17]:
ax = np.random.randint(10, size=(3,3))
with pd.HDFStore('abc' + '.h5', 'a') as h5f:
    print(dir(h5f))
    h5f.get_storer('/')
    h5f._handle['/f'] =  x
    

['__bytes__', '__class__', '__contains__', '__delattr__', '__delitem__', '__dict__', '__dir__', '__doc__', '__enter__', '__eq__', '__exit__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setitem__', '__sizeof__', '__str__', '__subclasshook__', '__unicode__', '__weakref__', '_check_if_open', '_complevel', '_complib', '_create_storer', '_filters', '_fletcher32', '_handle', '_mode', '_path', '_read_group', '_validate_format', '_write_to_group', 'append', 'append_to_multiple', 'close', 'copy', 'create_table_index', 'filename', 'flush', 'get', 'get_node', 'get_storer', 'groups', 'is_open', 'items', 'iteritems', 'keys', 'open', 'put', 'remove', 'root', 'select', 'select_as_coordinates', 'select_as_multiple', 'select_column']


TypeError: cannot create a storer if the object is not existing nor a value are passed

In [ ]:
!ls -alh test.msg

In [ ]:
from autocnet.utils.utils import make_homogeneous

In [ ]:
x = np.random.randint(100, size=(10,2))
make_homogeneous(x)

In [ ]:
df = pd.DataFrame(x, columns=['x', 'y'])
df

In [ ]:
make_homogeneous(df)

In [13]:
d = {}
serial = None
d['hash'] = serial
print(d)

{'hash': None}


In [14]:
serial = 123
print(d)

{'hash': None}


In [5]:
from uuid import uuid4

In [16]:
dir(uuid4())
uuid4().hex

'ac6dad2fb36d411b926ff0418b0091e9'